## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import (
    Concatenate, Lambda, Dense, Dropout, Activation, Flatten, LSTM, SpatialDropout2D, Conv2D, MaxPooling2D,
    AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, BatchNormalization, ReLU, Input
)
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import optimizers, regularizers, backend as K
from sklearn.metrics import (
    confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, classification_report,
    cohen_kappa_score, hamming_loss, log_loss, zero_one_loss, matthews_corrcoef, roc_curve, auc
)
from sklearn.preprocessing import MinMaxScaler, StandardScaler, label_binarize
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from tensorflow.keras.utils import to_categorical, plot_model
from time import time
import time as tm
import datetime
from skimage.util.shape import view_as_blocks
import glob
import os
import random
import ntpath
import copy
from tensorflow import keras
from tensorflow.keras import layers

from tfkan.layers import DenseKAN

## Filters and Dataset 

In [2]:
################################################## 30 SRM FILTERS
srm_weights = np.load('../SRM_Kernels.npy') 
biasSRM=np.ones(30)
print (srm_weights.shape)
################################################## TLU ACTIVATION FUNCTION
T3 = 3;
def Tanh3(x):
    tanh3 = K.tanh(x)*T3
    return tanh3
##################################################
def thtanh(x,t):
    th=K.tanh(x)*t
    return th

(5, 5, 1, 30)


S-UNIWARD BOSSbase 1.01 PAYLOAD = 0.4bpp

In [3]:
#Train
X_train = np.load('../data_gbras/X_train.npy')
y_train = np.load('../data_gbras/y_train.npy')
#Valid
X_valid = np.load('../data_gbras/X_valid.npy')
y_valid = np.load('../data_gbras/y_valid.npy')
#Test
X_test = np.load('../data_gbras/X_test.npy')
y_test = np.load('../data_gbras/y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

(12000, 256, 256, 1)
(12000, 2)
(4000, 256, 256, 1)
(4000, 2)
(6000, 256, 256, 1)
(4000, 2)


## Functions arquitecture

In [4]:
def squeeze_excitation_layer(input_layer, out_dim, ratio, conv):
  squeeze = tf.keras.layers.GlobalAveragePooling2D()(input_layer)
  excitation = tf.keras.layers.Dense(units=out_dim / ratio, activation='relu')(squeeze)
  excitation = tf.keras.layers.Dense(out_dim,activation='sigmoid')(excitation)
  excitation = tf.reshape(excitation, [-1,1,1,out_dim])
  scale = tf.keras.layers.multiply([input_layer, excitation])
  if conv:
    shortcut = tf.keras.layers.Conv2D(out_dim,kernel_size=1,strides=1,
                                      padding='same',kernel_initializer='he_normal')(input_layer)
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)
  else:
    shortcut = input_layer
  out = tf.keras.layers.add([shortcut, scale])
  return out



def sreLu (input):
  return ReLU(negative_slope=0.1, threshold=0)(input)

def sConv(input,parameters,size,nstrides):
  return Conv2D(parameters, (size,size), strides=(nstrides,nstrides),padding="same", kernel_initializer='glorot_normal', kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(input)

def sBN (input):
  return tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=True, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(input)

def sGlobal_Avg_Pooling (input):
  return tf.keras.layers.GlobalAveragePooling2D()(input)

def sDense (input, n_units, activate_c):
  return tf.keras.layers.Dense(n_units,activation=activate_c)(input)

def smultiply (input_1, input_2):
  return tf.keras.layers.multiply([input_1, input_2])

def sadd (input_1, input_2):
  return tf.keras.layers.add([input_1, input_2])

# Initial learning rate for the optimizer
initial_learning_rate = 0.001

# Learning rate schedule with exponential decay
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
  

### Blocks

In [5]:
def Block_1 (input, parameter):
  output = sConv(input, parameter, 3, 1)
  output = sBN(output)
  output = sreLu(output)
  return output
  


def SE_Block(input, out_dim, ratio):
  output = sGlobal_Avg_Pooling(input)
  output = sDense(output, out_dim/ratio, 'relu')
  output = sDense(output, out_dim, 'sigmoid')
  return output
  
  
  
def Block_2 (input, parameter):
  output = Block_1(input, parameter)
  output = sConv(output, parameter, 3, 1)
  output = sBN(output)
  multiplier = SE_Block(output,  parameter, parameter)
  # output = smultiply(output, output)
  output = smultiply(multiplier, output)
  output = sadd(output, input)
  return output
  
  

def Block_3 (input, parameter):
  addition = sConv(input, parameter, 1, 2)
  addition = sBN(addition)
  output = sConv(input, parameter, 3, 2)
  output = sBN(output)
  output = sreLu(output)
  output = sConv(output, parameter, 3, 1)
  output = sBN(output)
  multiplier = SE_Block(output,  parameter, parameter)
  output = smultiply(multiplier, output)
  output = sadd(output, addition)
  return output  
  
def Block_4 (input, parameter):
  output = Block_1(input, parameter)
  output = sConv(input, parameter, 3, 1)
  output = sBN(output)
  
  return output  

## Functions to training

In [8]:

def train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size, epochs, initial_epoch = 0, model_name="", num_test=""):
    start_time = tm.time()
    log_dir="D:/testing_"+num_test+"/"+model_name+"_"+"{}".format(time())
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir)
    filepath = log_dir+"/saved-model.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath, 
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1
    )
    model.reset_states()
    history=model.fit(X_train, y_train, epochs=epochs, 
                        callbacks=[tensorboard,  checkpoint], 
                        batch_size=batch_size,
                        validation_data=(X_valid, y_valid),
                        initial_epoch=initial_epoch)
    
    metrics = model.evaluate(X_test, y_test, verbose=0)
    results_dir="D:/testing_"+num_test+"/"+model_name+"/"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
      
        plt.figure(figsize=(10, 10))
        #plt.subplot(1,2,1)
        #Plot training & validation accuracy values
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Accuracy Vs Epochs')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'Accuracy_Xu_Net_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'Accuracy_Xu_Net_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'Accuracy_Xu_Net_'+model_name+'.pdf', format='pdf')
        plt.show()
        
        plt.figure(figsize=(10, 10))
        #plt.subplot(1,2,2)
        #Plot training & validation loss values
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Loss Vs Epochs')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'Loss_Xu_Net_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'Loss_Xu_Net_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'Loss_Xu_Net_'+model_name+'.pdf', format='pdf')
        plt.show()

    TIME = tm.time() - start_time
    print("Time "+model_name+" = %s [seconds]" % TIME)
    return {k:v for k,v in zip (model.metrics_names, metrics)}

## CVKAN

In [80]:
def new_arch_kan():
  tf.keras.backend.clear_session()
  inputs = tf.keras.Input(shape=(256,256,1), name="input_1")
  #Layer 1
  layers_ty = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), padding='same', trainable=False, activation=Tanh3, use_bias=True)(inputs)
  layers_tn = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), padding='same', trainable=True, activation=Tanh3, use_bias=True)(inputs)

  layers = tf.keras.layers.add([layers_ty, layers_tn])
  layers1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
  #Layer 2
  
  # L1
  layers = Block_1(layers1,64)

  # L2
  layers = Block_1(layers,64)

  # L3 - L7
  for i in range(5):
    layers = Block_2(layers,64)

  # L8 - L11
  for i in [64, 64, 128, 256]:
    layers = Block_3(layers,i)

  # L12
  layers = Block_4(layers,512)
  #CVT=Transform_sh_1(layers)

  representation = tf.keras.layers.LayerNormalization(epsilon=LAYER_NORM_EPS_2)(layers)
  representation = tf.keras.layers.GlobalAvgPool2D()(representation)
  #---------------------------------------------------Fin de Transformer 2------------------------------------------------------------------------#
  # Classify outputs.
      #FC
  layers = DenseKAN(64)(representation)
  layers = DenseKAN(32)(layers)
  layers = DenseKAN(16)(layers)

  #Softmax
  layers = DenseKAN(2)(layers)
  predictions = tf.keras.layers.Softmax(axis=1)(layers)
  model =tf.keras.Model(inputs = inputs, outputs=predictions)

  # Compile the model if the compile flag is set
  if compile:
      model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

      print("Transformer_create")

  return model


model2 = new_arch_kan()  
  

Transformer_create


## Training

In [54]:
base_name="04S-UNIWARD"
name="Model_"+'CVkan_prueba1'+"_"+base_name
_, history  = train(model2, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size=8, epochs=400, model_name=name, num_test='1_cvt')

Epoch 1/400


ResourceExhaustedError: Graph execution error:

Detected at node 'model/batch_normalization_10/FusedBatchNormV3' defined at (most recent call last):
    File "c:\Users\sergioa.holguin\.conda\envs\machine\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\sergioa.holguin\.conda\envs\machine\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\sergioa.holguin\.conda\envs\machine\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\sergioa.holguin\.conda\envs\machine\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\sergioa.holguin\.conda\envs\machine\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\sergioa.holguin\.conda\envs\machine\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      